In [26]:
# Import necessary libraries
import requests
import json

# Define the FastAPI endpoints
PREDICT_ENDPOINT = "http://127.0.0.1:8000/predict"
STORE_TWEET_ENDPOINT = "http://127.0.0.1:8000/store_tweet"

In [27]:
# Function to predict if a tweet is hate speech
def predict_tweet(text, user):
    response = requests.post(PREDICT_ENDPOINT, json={"text": text, "user": user})
    if response.status_code == 200:
        return response.json()
    else:
        print("Prediction failed:", response.json())
        return None


In [28]:
# Function to store a tweet if it's safe to post
def store_tweet(tweet_id, user_id, retweet_id=None):
    data = {
        "tweet_id": tweet_id,
        "retweet_id": retweet_id,
        "user_id": user_id
    }
    response = requests.post(STORE_TWEET_ENDPOINT, json=data)
    if response.status_code == 200:
        return response.json()
    else:
        print("Storing tweet failed:", response.json())
        return None

In [29]:
# Example tweet text and user
tweet_text = "This is an example tweet for testing purposes."
user_id = "example_user"

In [17]:
#Predict if the tweet is hate speech
prediction_result = predict_tweet(tweet_text, user_id)

if prediction_result:
    print("Prediction Result:", prediction_result)
    
    # Check if the tweet is safe to post (logreg_result should be 0)
    if prediction_result["logreg_result"] == 0:
        tweet_id = prediction_result["tweet_id"]
        
        # Step 3: Store the tweet in the posted tweets database
        store_result = store_tweet(tweet_id, user_id)
        if store_result:
            print("Tweet successfully posted:", store_result)
    else:
        print("Tweet is considered hate speech and will not be posted.")
else:
    print("Prediction failed.")

Prediction Result: {'tweet_id': '181512ed', 'tweet': 'This is an example tweet for testing purposes.', 'user': 'example_user', 'likes': 0, 'retweets': 0, 'logreg_prob': 0.01582927467824479, 'logreg_result': 0, 'cnn_prob': None, 'cnn_result': None}
Tweet successfully posted: {'tweet_id': '181512ed', 'retweet_id': None, 'text': 'This is an example tweet for testing purposes.', 'retweets': 0, 'id': 1, 'user_id': 'example_user', 'likes': 0, 'safety_status': None}


## create random users, and post tweets under test.csv under their names

In [30]:
import pandas as pd
import random
import requests
from datetime import datetime, timezone

base_url = "http://localhost:8000"

# Define users
users = [f"user{i}" for i in range(1, 9)]

In [54]:
# Read tweets from CSV
test_csv_path = '../data/test.csv'
df = pd.read_csv(test_csv_path)

# Shuffle tweets and split into 8 equal parts
tweets = df['tweets'].tolist()
random.shuffle(tweets)
tweets = tweets[43:47]
num_tweets_per_user = len(tweets) // len(users)

In [55]:
num_tweets_per_user

1

In [56]:
# Function to post a tweet for prediction
def post_tweet_for_prediction(tweet, user):
    response = requests.post(f'{base_url}/predict', json={"text": tweet, "user": user})
    return response.json()

In [57]:
def store_posted_tweet(tweet_id, retweet_id, user_id, text, likes, retweets, safety_status, created_at):
    data = {
        "tweet_id": tweet_id,
        "retweet_id": retweet_id,
        "user_id": user_id,
        "text": text,
        "likes": likes,
        "retweets": retweets,
        "safety_status": safety_status,
        "created_at": created_at
    }
    response = requests.post(f'{base_url}/store_tweet', json=data)
    return response.json()

In [58]:
# Function to simulate user submissions
def simulate_user_submission():
    for i, user in enumerate(users):
        user_tweets = tweets[i * num_tweets_per_user : (i + 1) * num_tweets_per_user]
        for tweet in user_tweets:
            prediction = post_tweet_for_prediction(tweet, user)
            print("Prediction Result:", prediction)
            
            if prediction["logreg_result"] != 1:
                created_at = datetime.now(timezone.utc).isoformat()  # Convert datetime to ISO format string

                store_response = store_posted_tweet(
                    tweet_id=prediction["tweet_id"],
                    retweet_id=None,
                    user_id=user,
                    text=prediction["tweet"],
                    likes=prediction["likes"],
                    retweets=prediction["retweets"],
                    safety_status=None,  # Initial safety status is None
                    created_at=created_at
                )
                print("Store Tweet Response:", store_response)

In [59]:
# Run the simulation
simulate_user_submission()

Prediction Result: {'tweet_id': '9d7d2576', 'tweet': 'love island think there might be a guy in the mix for zara eek lucky girl', 'user': 'user4', 'likes': 0, 'retweets': 0, 'logreg_prob': 0.404016058196919, 'logreg_result': 0, 'cnn_prob': None, 'cnn_result': None, 'created_at': '2024-07-25T08:25:05.511616'}
Store Tweet Response: {'id': 21, 'user_id': 'user4', 'likes': 0, 'safety_status': None, 'text': 'love island think there might be a guy in the mix for zara eek lucky girl', 'tweet_id': '9d7d2576', 'retweet_id': None, 'retweets': 0, 'created_at': '2024-07-25T08:25:05.511616'}
Prediction Result: {'tweet_id': '71d24af1', 'tweet': 'cemu toledo yo lonnie momentary upravo your bog... taking little primitive as stave see pride go i', 'user': 'user2', 'likes': 0, 'retweets': 0, 'logreg_prob': 0.9680542198318599, 'logreg_result': 1, 'cnn_prob': None, 'cnn_result': None, 'created_at': '2024-07-25T08:25:09.640818'}
Prediction Result: {'tweet_id': '8f3bf7a5', 'tweet': 'this arose time to talk 

## random users create random tweets 

In [3]:
import pandas as pd
import random
import requests
import time
from datetime import datetime, timezone, timedelta

base_url = "http://localhost:8000"


In [16]:
# Define users
users = [f"user{i}" for i in range(1, 9)]

# Read tweets from CSV
test_csv_path = '../data/test.csv'
df = pd.read_csv(test_csv_path)

# df = df[df['labels']==1]
# Shuffle tweets and select 50 tweets
tweets = df['tweets'].tolist()
random.shuffle(tweets)
tweets = tweets[:20]


In [17]:

# Function to post a tweet for prediction
def post_tweet_for_prediction(tweet, user):
    response = requests.post(f'{base_url}/predict', json={"text": tweet, "user": user})
    return response.json()

def store_posted_tweet(tweet_id, retweet_id, user_id, text, likes, retweets, safety_status, created_at):
    data = {
        "tweet_id": tweet_id,
        "retweet_id": retweet_id,
        "user_id": user_id,
        "text": text,
        "likes": likes,
        "retweets": retweets,
        "safety_status": safety_status,
        "created_at": created_at
    }
    response = requests.post(f'{base_url}/store_tweet', json=data)
    return response.json()


In [21]:

# Function to simulate user submissions
def simulate_user_submission():
    total_tweets = 50
    tweet_count = 0
    end_time = datetime.now(timezone.utc) + timedelta(hours=1)
    
    while tweet_count < total_tweets and datetime.now(timezone.utc) < end_time:
        # Randomly select number of tweets to post (1 to 3)
        num_tweets_to_post = random.randint(1, 3)
        print(f"Create {num_tweets_to_post} tweets.")
        # Randomly select tweets for this batch
        if tweet_count + num_tweets_to_post > total_tweets:
            num_tweets_to_post = total_tweets - tweet_count  # Adjust to post remaining tweets

        user_tweets = random.sample(tweets, num_tweets_to_post)
        
        for tweet in user_tweets:
            user = random.choice(users)
            print(f"By user {user}.")
            prediction = post_tweet_for_prediction(tweet, user)
            print("Prediction Result:", prediction)

            if prediction["logreg_result"] != 1:
                created_at = datetime.now(timezone.utc).isoformat()  # Convert datetime to ISO format string

                store_response = store_posted_tweet(
                    tweet_id=prediction["tweet_id"],
                    retweet_id=None,
                    user_id=user,
                    text=prediction["tweet"],
                    likes=prediction["likes"],
                    retweets=prediction["retweets"],
                    safety_status=None,  # Initial safety status is None
                    created_at=created_at
                )
                print("Store Tweet Response:", store_response)

        tweet_count += num_tweets_to_post
        print("Batch post complete.")
        # Wait for a random interval between 5 and 10 seconds before posting next batch
        time.sleep(random.randint(15, 20))



In [22]:
# Run the simulation
simulate_user_submission()

Create 1 tweets.
By user user2.
Prediction Result: {'tweet_id': '6d83975f7d9afdabe8b68b1d95628e6c5b3817a6b95931fec65e8511630457b6', 'tweet': 'many women just are not about', 'user': 'user2', 'likes': 0, 'retweets': 0, 'logreg_prob': 0.9051479096294579, 'logreg_result': 0, 'cnn_prob': None, 'cnn_result': None, 'created_at': '2024-07-25T09:34:52.283379'}
Store Tweet Response: {'user_id': 'user2', 'id': 23, 'likes': 0, 'safety_status': None, 'tweet_id': '6d83975f7d9afdabe8b68b1d95628e6c5b3817a6b95931fec65e8511630457b6', 'retweet_id': None, 'text': 'many women just are not about', 'retweets': 0, 'created_at': '2024-07-25T09:34:52.283379'}
Batch post complete.
Create 2 tweets.
By user user4.
Prediction Result: {'tweet_id': '434e5197ef77bfb05e79cb0cdb044c72a53d975dd435b9b082bbbb35e29bf6dc', 'tweet': 'a third defendant the jew gary german of brooklyn remains on the loose', 'user': 'user4', 'likes': 0, 'retweets': 0, 'logreg_prob': 0.7136924815147347, 'logreg_result': 0, 'cnn_prob': None, 'cnn

KeyboardInterrupt: 

## 3 users to report tweets

In [13]:
tweet_ids = ["28f43075", "4bce7cdc", "00a628267ee60cf699de81025777fe51b87f320cab204bebb1d7ba3017b4c1eb", "81de6805"]
users = ["user4", "user2", "user6", "user1"]

def report_tweet(report_data):
    url = f"{base_url}/report_tweet"
    response = requests.post(url, json=report_data)
    
    if response.status_code == 200:
        print("Report submitted successfully.")
    else:
        print(f"Failed to submit report: {response.status_code}")
        print(f"Response: {response.json()}")

In [14]:
for i in range(4):
    report_data = {
        "tweet_id": tweet_ids[i],
        "user_id": users[i],
        "safety_status": 1,
        "reported_at": datetime.now(timezone.utc).isoformat()
    }
    report_tweet(report_data)

Report submitted successfully.
Report submitted successfully.
Report submitted successfully.
